In [5]:
# import logging
import json
import re

# JSON formatting functions
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r', encoding="utf8") as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data['content'].replace("\n", " ")
        
        entities = []
        data_annotations = data['annotation']
        if data_annotations is not None:
            for annotation in data_annotations:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    point_start = point['start']
                    point_end = point['end']
                    point_text = point['text']

                    lstrip_diff = len(point_text) - len(point_text.lstrip())
                    rstrip_diff = len(point_text) - len(point_text.rstrip())
                    if lstrip_diff != 0:
                        point_start = point_start + lstrip_diff
                    if rstrip_diff != 0:
                        point_end = point_end - rstrip_diff
                    entities.append((point_start, point_end + 1 , label))
        training_data.append((text, {"entities" : entities}))
    return training_data

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append((valid_start, valid_end, label))
        cleaned_data.append((text,valid_entities))
    return cleaned_data


In [6]:
data = trim_entity_spans(convert_dataturks_to_spacy("Entity Recognition in Resumes.json"))

# data1=convert_dataturks_to_spacy("Entity Recognition in Resumes.json")
# data2=preprocessing(data)
# data=trim_entity_spans(data)
    
# # print(data[0])
# # print()
# # print()
# print(data1[0])
# print()
# print()
# print(data2[0])
# print()
# print(data1[0][0][316:349])
# print(data2[0][0][372:405])
len(data)
# data

220

In [7]:
def SelfAnnoted(Path):
    Cleaned=[]
    
    with open(Path, 'r', encoding="utf8") as f:
            lines = f.readlines()

    for line in lines:
        sample = json.loads(line)
        annot=sample["annotations"]
        text=annot[0][0]
        text= re.sub(r'[\s]'," ", text)
        entity=annot[0][1]["entities"]
        Cleaned.append((text,entity))
    return Cleaned
data2=SelfAnnoted("Self_Annot.json")    
data=data+data2
data[234]
len(data)

267

In [3]:
# def clean_entities(training_data):
    
#     clean_data = []
#     for text, annotation in training_data:
        
#         entities = annotation.get('entities')
#         entities_copy = entities.copy()
        
#         # append entity only if it is longer than its overlapping entity
#         i = 0
#         for entity in entities_copy:
#             j = 0
#             for overlapping_entity in entities_copy:
#                 # Skip self
#                 if i != j:
#                     e_start, e_end, oe_start, oe_end = entity[0], entity[1], overlapping_entity[0], overlapping_entity[1]
#                     # Delete any entity that overlaps, keep if longer
#                     if ((e_start >= oe_start and e_start <= oe_end) \
#                     or (e_end <= oe_end and e_end >= oe_start)) \
#                     and ((e_end - e_start) <= (oe_end - oe_start)):
#                         entities.remove(entity)
#                 j += 1
#             i += 1
#         clean_data.append((text, {'entities': entities}))
                
#     return clean_data

# data = clean_entities(data)

In [4]:
# # print(data[5])

# def function(data):
#     processed=[]
#     for sample in data:
#         Merge=[]
#         text=sample[0]
#         entities=sample[1]["entities"]
#         Start=entities[0][0]
#         End=entities[0][1]
#         Label=entities[0][2]
#         temp_label=""
#         for entity in entities:
#             if(Label==entity[2]):
#                 Start=entity[0]
#                 temp_label=Label
#                 continue;
#             else:
#                 Merge.append((Start,End,temp_label))
#                 Start=entity[0]
#                 End=entity[1]
#                 Label=entity[2]
#                 temp_label=entity[2]
                             
         
#         processed.append((text, {'entities': Merge}))
         
#     return processed
# data=function(data)
                
    
# data[50]

In [5]:
# !pip install spacy_transformers


In [8]:
# !python -m spacy download en_core_web_lg
# spacy.__vers|ion__
len(data)


267

In [9]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

def get_spacy_doc(data):
    nlp = spacy.blank("en") # load a new spacy model
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(data): # data in previous format
        doc = nlp(text) # create doc object from text
#         annot=annot["entities"]
        ents=[]
        entity_indices=[]
        for start, end, label in annot :
            # add character indexes
            skip_entity=False
            for idx in range(start,end):
                if idx in entity_indices:
                    skip_entity=True
                    break
            if skip_entity==True:
                continue
            entity_indices=entity_indices+list(range(start,end))
            
            
            try:
                span = doc.char_span(start, end, label=label)
            except:
                continue
            if span is None:
#                 err_data=str([start,end])+"   "+str(text)+"\n"
#                 file.write(err_data)
                continue
            else:
                ents.append(span)
            
        try:
            doc.ents=ents
            db.add(doc)
        except:
            pass
    return db
#     print("DATA CHANGE--------------------------------------------")
db=get_spacy_doc(data)
db.to_disk("./train.spacy") # save the docbin object

100%|████████████████████████████████████████| 267/267 [00:01<00:00, 144.61it/s]


In [10]:
# from sklearn.model_selection import train_test_split
# train,test=train_test_split(data,test_size=0.3)


In [18]:
# # file=open("error.txt","w")
# db=get_spacy_doc(train)
# db.to_disk("train.spacy")
# db=get_spacy_doc(test)
# db.to_disk("test.spacy")


In [10]:
!python -m spacy init fill-config base_config.cfg config.cfg



✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [12]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy --gpu-id 0
# !python -m pip --version
# !python -m pip install --upgrade pip

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-11-25 13:36:23,439] [INFO] Set up nlp object from config
[2022-11-25 13:36:23,445] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-11-25 13:36:23,448] [INFO] Created vocabulary
[2022-11-25 13:36:23,839] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
[2022-11-25 13:36:23,840] [INFO] Added vectors: en_core_web_sm
[2022-11-25 13:36:23,840] [INFO] Finished initializing nlp object
[2022-11-25 13:36:36,649] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0

In [49]:
# !pip install spacy==2.1.4

In [76]:
# def train_model(train_data):
#     nlp = spacy.blank('en')
#     if 'ner' not in nlp.pipe_names:
# #         ner = nlp.create_pipe('ner')
#         ner=nlp.add_pipe("ner")
    
#     for _, annotation in train_data:
#         for ent in annotation['entities']:
#             ner.add_label(ent[2])
            
    
#     other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
#     with nlp.disable_pipes(*other_pipes):  # only train NER
#         optimizer = nlp.begin_training()
#         for itn in range(10):
#             print("Statring iteration " + str(itn))
#             random.shuffle(train_data)
#             losses = {}
#             index = 0
#             for text, annotations in train_data:
#                 try:
#                     nlp.update(
#                         [text],  # batch of texts
#                         [annotations],  # batch of annotations
#                         drop=0.2,  # dropout - make it harder to memorise data
#                         sgd=optimizer,  # callable to update weights
#                         losses=losses)
#                 except Exception as e:
#                     pass
# #                 
#             print(losses)

In [74]:
# !python -m spacy download en_core_web_lg

In [5]:
# train_model(data)

In [54]:
# ###
# nlp.to_disk('nlp_model')
# nlp_model = spacy.load('nlp_model')

In [7]:
#####
# doc = nlp_model(data[0][0])
# for ent in doc.ents:
#     print(f'{ent.label_.upper():{30}}- {ent.text}')
    
data[216]

("Anil Kumar Microsoft Azure (Basic Management)  Delhi, Delhi - Email me on Indeed: indeed.com/r/Anil-Kumar/96983a9dd7222ae5  Seeking a challenging career as an IT-Manager utilizes my analytical skills experience in design and development area of IT development & management. Long term objective is to achieve an IT consultant position.  STRENGTH:  Ability of solving problems independently, positive approach, quick learner and good performer both in team & independent environment, hardworking self-confident, sincere and deterministic.  WORK EXPERIENCE  Microsoft Azure (Basic Management)  FTP and TELNET -  2008 to 2008  • Design & implementation of Servers, Exchange Servers and Network according to the customer specific needs. • Designing and implementations of LAN /WAN, protocols used like TCP /IP, UDP, DHCP, HTTP, FTP and TELNET. • Providing remote support for maintaining Pentium, Pentium II, Pentium III, Servers on Windows NT, Windows 2000 Server, MS Windows Server 2003 R2 and MS Windo

In [19]:
# !python -m spacy init fill-config base_config.cfg config.cfg
data[230]

("Abhijeet Manhas Email : abhijeetmanhas720@gmail.com github.com/abhijeetmanhas Mobile : +91-862-896-3924 twitter.com/astromanhas linkedin.com/abhimanhas Education \\x0fIndian Institute of Technology Mandi Mandi, India B.Tech in Computer Science Engineering; CGPA: 8.41 July 2018 { Present \\x0eRelevant coursework : Data Structures and Algorithms, Computing and Data Science, Data Science II, Data Science III, Applied Databases Practicum, Information and Database Systems, Linear Algebra. \\x0ePositions of Responsibility : Coordinator of Space Technology and Astronomy Cell, IIT Mandi for academic year 2019-20. Volunteered as Publicity head of AstraX 2020 (annual inter-college Astro-meet). Experience \\x0fGoogle Summer of Code Developer SunPy, OpenAstronomy Metadata Searches using Fido May 2020 - Present \\x0eWorking with SOAP and Web-Scrapers : Created methods to retrieve more details from XML responses received from querying Virtual Solar Observatory using Zeep Client. Build scrapers to 

In [21]:
# !python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

In [49]:
nlp = spacy.load(r"./output/model-best") #load the best model

In [50]:
doc = nlp1(data[230][0]) # input sample text
for ent in doc.ents:
    print("-----------------------------------------------------------------------------------------------------------------")
    
        
    print(ent.text,"-------->>>>>>>",ent.label_)

-----------------------------------------------------------------------------------------------------------------
Abhijeet Manhas -------->>>>>>> Name
-----------------------------------------------------------------------------------------------------------------
Data Structures and -------->>>>>>> Skills
-----------------------------------------------------------------------------------------------------------------
Data Science -------->>>>>>> Skills
-----------------------------------------------------------------------------------------------------------------
Python -------->>>>>>> Skills
-----------------------------------------------------------------------------------------------------------------
Dart -------->>>>>>> Skills
-----------------------------------------------------------------------------------------------------------------
C++ -------->>>>>>> Skills
-----------------------------------------------------------------------------------------------------------------
C

In [51]:
from spacy import displacy
doc = nlp1(data[230][0])
displacy.render(doc,style="ent",jupyter=True)

In [43]:
# importing required modules
import PyPDF2

# creating a pdf file object
pdfFileObj = open('sample_input.pdf', 'rb')

# creating a pdf reader object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# printing number of pages in pdf file
print(pdfReader.numPages)
text_sample=""

# creating a page object
for i in range(pdfReader.numPages):
    pageObj = pdfReader.getPage(i)
    text_sample=text_sample+pageObj.extractText()

    # extracting text from page
    print(pageObj.extractText())

# closing the pdf file object
pdfFileObj.close()


3
Dushyant Bhatt
BI / Big Data/ Azure
Hyderabad-Deccan, T elangana, T elangana -Email me on Indeed: indeed.com/r/Dushyant-
Bhatt/140749dace5dc26f
• 10+ years of Experience in Designing, Development, Administration, Analysis, Management in
the Business Intelligence Data warehousing, Client Server T echnologies, Web-based Applications,
cloud solutions and Databases.
• Data warehouse: Data analysis, star/ snow flake schema data modeling and design specific to
data warehousing and business intelligence environment.
• Database: Experience in database designing, scalability, back-up and recovery, writing and
optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. 
• Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream
Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL).
• Big Data: Worked Azure data lake store/analytics for big data processing and Azure data factory
to schedul

In [44]:
#Function to extract Phone Numbers from string using regular expressions
def extract_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = r.findall(string)
    return [re.sub(r'\D', '', number) for number in phone_numbers]
#Function to extract Email address from a string using regular expressions
def extract_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)

Pho=extract_phone_numbers(text_sample)
E_mail=extract_email_addresses(text_sample)

In [45]:
text_sample=text_sample.replace("\n"," ")
text_sample=re.sub(r"/[^a-zA-Z0-9\-\+\.\/\@]+"," ",text_sample)

import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

text = en(text_sample)

lst=[]
for token in text:
    if token not in stopwords:    #checking whether the word is not 
        lst.append(token)                    #present in the stopword list.
        
#Join items in the list
# print("Original text  : ",text)
# # print("Text after removing stopwords  :   ",' '.join(lst))
# text_sample1=" ".join(lst)
# text_sample1=text_sample1
text_sample

'Dushyant Bhatt BI  Big Data Azure Hyderabad-Deccan, T elangana, T elangana -Email me on Indeed: indeed.com/r/Dushyant- Bhatt/140749dace5dc26f • 10+ years of Experience in Designing, Development, Administration, Analysis, Management in the Business Intelligence Data warehousing, Client Server T echnologies, Web-based Applications, cloud solutions and Databases. • Data warehouse: Data analysis, star snow flake schema data modeling and design specific to data warehousing and business intelligence environment. • Database: Experience in database designing, scalability, back-up and recovery, writing and optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.  • Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL). • Big Data: Worked Azure data lake store/analytics for big data processing and Azure data factory to schedule U-

In [46]:
from spacy import displacy
doc = nlp1(text_sample)
displacy.render(doc,style="ent",jupyter=True)

In [65]:
import math
from collections import Counter

A=["C++","Java","Python","SQL","Database","web development"]
B=["SQL","Database","web development","MySQL"]

A = Counter(A)
B = Counter(B)

def cosine_similarity(Skills,Obt_Skills):
    Skills=Counter(Skills)
    obt_Skills=Counter(Obt_Skills)
    terms = set(Skills)
    dotprod = sum(Skills.get(k, 0) * Obt_Skills.get(k, 0) for k in terms)
    magA = math.sqrt(sum(Skills.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(Obt_Skills.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)
cosine_similarity(A,B)

0.7071067811865476

In [59]:
def getSkills(nlp,data):
    doc=nlp(text)
    sk=[]
    for ent in doc.ents:
        if ent.label_=="Skills":
            sk.append(ent.text)
    return sk

In [60]:
Computer_Science_Skills=["C++","Java","Python","SQL","Database","web development","MySQL","Oracle","Cloud Computing","Problem-solving","Communication","Javascript","UNIX","Linux","Java"]
Data_Science_Skills=["Python","Machine Learning","Deep Learning","Tableau","Data Visualisation","Big Data","Data Mining","Natural Language Processing","NLP","TensorFlow","Statistical Modeling","Data Science","Web Scraping"]

In [80]:
def Skills(n,Profile,nlp):
    Skill_Score={}
    
    for i in range(1,n):
        pdfFileObj = open(str(i)+".pdf", 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        text_sample=""
        for i in range(pdfReader.numPages):
            pageObj = pdfReader.getPage(i)
            text_sample=text_sample+pageObj.extractText()
        pdfFileObj.close()
        
        text_sample=text_sample.replace("\n"," ")
        text_sample=re.sub(r"[^a-zA-Z0-9\-\+\.\/\@]"," ",text_sample)
        doc = nlp(text_sample)
        displacy.render(doc,style="ent",jupyter=True)
        print("-----------------------------------------------------------------------------------------------")
#         Skill=getSkills(nlp,text_sample)
#         Skill_Score[i]=cosine_similarity(Profile,Skill)
    return None

In [81]:
print(Skills(7,Computer_Science_Skills,nlp))

-----------------------------------------------------------------------------------------------


-----------------------------------------------------------------------------------------------


-----------------------------------------------------------------------------------------------


-----------------------------------------------------------------------------------------------


-----------------------------------------------------------------------------------------------


-----------------------------------------------------------------------------------------------
None


In [38]:
str(1)+".pdf"=="1.pdf"

True